Paint con manos

In [1]:
import cv2
import mediapipe as mp
from pydub import AudioSegment
import pygame

In [12]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

pygame.mixer.init()

sound_files = {
    'neutral': "C:/Users/PC Gaming/Downloads/tomp3.cc - Gaby Music Lunay Luar La L  No Te Quieren Conmigo Official Video.mp3",
    'top': "C:/Users/PC Gaming/Downloads/tomp3.cc - Waterfall Sound Effects.mp3",
    'bottom': "C:/Users/PC Gaming/Downloads/tomp3.cc - Fire Sound Effect.mp3",
    'left': "C:/Users/PC Gaming/Downloads/tomp3.cc - STONE FALLING sound effect.mp3",
    'right': "C:/Users/PC Gaming/Downloads/tomp3.cc - Wind Sound FX.mp3",
    'top_left': "C:/Users/PC Gaming/Downloads/tomp3.cc - FIRE WHOOSH TRANSITION SOUND EFFECTS.mp3",
    'top_right': "C:/Users/PC Gaming/Downloads/tomp3.cc - Walking on mud sound effect  footsteps muddy SOUND EFFECT FREE Audio HQ WAV.mp3",
    'bottom_left': "C:/Users/PC Gaming/Downloads/tomp3.cc - Lightning Storm Sound Effect.mp3",
    'bottom_right': "C:/Users/PC Gaming/Downloads/tomp3.cc - Lava  Sound Effect.mp3"
}

elements = {
    'neutral': "neutral",
    'water': "water",
    'fire': "fire",
    'earth': "earth",
    'wind': "wind",
    'lightning': "lightning",
    'lava': "lava",
    'mud': "mud",
    'superfire': "superfire"
}

cap = cv2.VideoCapture(0)
cap.set(3, 960)
cap.set(4, 720)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

                index_x = hand_landmarks.landmark[8].x
                index_y = hand_landmarks.landmark[8].y

                pygame.mixer.music.load(sound_files['neutral'])

                # Map the hand position to color values and play sound
                if index_y < 0.2 and index_x > 0.3 and index_x < 0.6:
                    color = (255, 0, 0)  # Blue for top
                    pygame.mixer.music.load(sound_files['top'])
                    element = elements['water']
                elif index_y > 0.8 and index_x > 0.3 and index_x < 0.6:
                    color = (0, 0, 255)  # Red for bottom
                    pygame.mixer.music.load(sound_files['bottom'])
                    element = elements['fire']
                elif index_x < 0.2 and index_y > 0.3 and index_y < 0.6:
                    color = (0, 64, 255)  # Brown for left
                    pygame.mixer.music.load(sound_files['left'])
                    element = elements['earth']
                elif index_x > 0.8 and index_y > 0.3 and index_y < 0.6:
                    color = (255, 255, 255)  # White for right
                    pygame.mixer.music.load(sound_files['right'])
                    element = elements['wind']
                elif index_x < 0.3 and index_y < 0.3:
                    color = (0, 255, 0)  # Green for top-left
                    pygame.mixer.music.load(sound_files['top_left'])
                    element = elements['superfire']
                elif index_x >= 0.3 and index_y < 0.3:
                    color = (255, 0, 255)  # Red for top-right
                    pygame.mixer.music.load(sound_files['top_right'])
                    element = elements['mud']
                elif index_x < 0.3 and index_y >= 0.3:
                    color = (255, 255, 0)  # SaddleBrown for bottom-left
                    pygame.mixer.music.load(sound_files['bottom_left'])
                    element = elements['lightning']
                elif index_x >= 0.3 and index_y >= 0.3:
                    color = (0, 127, 255)  # Yellow for bottom-right
                    pygame.mixer.music.load(sound_files['bottom_right'])
                    element = elements['lava']
                
                if cv2.waitKey(1) & 0xFF == ord('f') or cv2.waitKey(1) & 0xFF == ord('F'):
                    pygame.mixer.music.play()

                overlay = image.copy()
                overlay[:] = color 
                cv2.addWeighted(overlay, 0.75, image, 0.25, 0, image)

                # Set font parameters
                font = cv2.FONT_HERSHEY_SIMPLEX
                fontScale = 1
                thickness = 2
                pos = (50, 50)

                cv2.putText(image, str(element), pos, font, fontScale, (0, 0, 0), thickness, cv2.LINE_AA)

        cv2.imshow('MediaPipe Hands', image)

        if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(1) & 0xFF == ord('Q'):
            break
            

cap.release()
cv2.destroyAllWindows()